In [1]:
# PTT title generator from content base on seq2seq model.
import pandas as pd
import time
import csv
import os
import types 

#Initialize for files name and path.
base_dir = 'C:/Users/User/Raw data/PTT'
totalboard_name = 'TJ-BG-AT-SX-CC-MV'
board_name = ['Tech_job','Boy-Girl','AllTogether','sex','C_Chat','movie']
board_dict_index = {'Tech_job':0,'Boy-Girl':1,'AllTogether':2,'sex':3,'C_Chat':4,'movie':5}
max_word_length = 256
min_word_length = 100
max_title_length = 20
label_newnum = len(board_name)

new_content_dir = []
totalboard_dir = os.path.join(base_dir, totalboard_name)
if not os.path.exists(totalboard_dir):
    os.makedirs(totalboard_dir)
for name_ind in range(len(board_name)):
    # New data path
    new_content_dir.append(os.path.join(totalboard_dir, board_name[name_ind]+'_content'))
    if not os.path.exists(new_content_dir[name_ind]):
        os.makedirs(new_content_dir[name_ind])

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import jieba

# Import dictionary fron jieba and PTT.
jieba.set_dictionary('C:/Users/User/Anaconda3/Lib/site-packages/jieba/dict.txt.big')
jieba.load_userdict('C:/Users/User/Anaconda3/Lib/site-packages/jieba/userdict.txt')

text_content = []
text_label = []
text_title = []

GO_word = '<GO>' #解碼器端的句子起始標識符。
EOS_word = '<EOS>' #解碼器端的句子結束標識符。
PAD_word = '<PAD>' #補全字符。
UNK_word = '<UNK>' #低頻詞或者一些未遇到過的詞等。

# Load PTT content and information.
#dfs_total = pd.read_csv(os.path.join(totalboard_dir, totalboard_name + '-test.csv'))
dfs_total = pd.read_csv(os.path.join(totalboard_dir, totalboard_name + '.csv'))
len_total = len(dfs_total)

for dfs_index in range(len_total):
    read_index = int(dfs_total.iloc[dfs_index, 0])
    type_name = str(dfs_total.iloc[dfs_index, 1])
    word_length = int(dfs_total.iloc[dfs_index, 5])
    title_name = str(dfs_total.iloc[dfs_index, 9])
    this_board_name = str(dfs_total.iloc[dfs_index, 10])
    
    # Add start and end character.
    text = GO_word + ' '
    with open(os.path.join(new_content_dir[board_dict_index[this_board_name]], str(read_index) + '.csv'),
                'r', encoding = 'utf-8-sig') as file:
        csvCursor = csv.reader(file)
        for rows in csvCursor:
            for row in rows:
                # Read content and remove empty.
                text = text + row
        else:
            text = text + ' ' + EOS_word
    
    # Split titles to words by Jieba.
    title_name.encode('utf-8-sig')
    title_jieba = jieba.cut(title_name, cut_all=False)
    title_name = GO_word
    word_length = 0
    for word in title_jieba:
        word_length += 1
        title_name = title_name + ' ' + word
    else:
        title_name = title_name + ' ' + EOS_word

    text_content.append(text)
    text_label.append(board_dict_index[this_board_name])
    text_title.append(title_name)
    file.close()

# Build vocabulary and convert content to sequence by Keras tool.
words_limit = 60000
tokenizer = Tokenizer(num_words=words_limit, 
                      filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n　，。！：；、？﹝﹞「」『』（）｛｝［］【】《》“”‘’＼｜〝〞‵′＋－＊／＝≦≧＿＠＃＄％︿＆～§◎．※ㄧ↔│○●☆★◇◆□■▽▼△▲㊣⊙⊕ˍ…﹌﹋﹎﹍﹉﹊‥–↑↓←→↖↗↙↘∥∕℅≒≡∩∪∞￣＿◤◥◣◢∵∴〒⊥∠⊿┼┴┬┤├▔─│▕┌┐└┘╭╮╰╯═╞╪╡╔╦╗╠╬╣╚╩╝╒╤╕╘╧╛╓╥╖╟╫╢╙╨╜║▓╱╲╳▁▂▄▅▆▇█▏▎▍▌▋▊▉▁▔', 
                      lower=False,
                      split=" ")
                      #oov_token=UNK_word)
tokenizer.fit_on_texts(text_content+text_title)
vocab = tokenizer.word_index
vocab_counts = tokenizer.word_counts
vocab_docs = tokenizer.word_docs

print(len(vocab))
anti_vocab = {}
vocab_keys = list(vocab.keys())
for vocab_word in vocab:
    vocab_index = vocab[vocab_word]
    anti_vocab[vocab_index] = vocab_keys[vocab_index-1]
    if vocab_index % 1000 == 0 or vocab_index < 20:
        print(vocab[vocab_word], vocab_word, anti_vocab[vocab_index], vocab_counts[vocab_word], vocab_docs[vocab_word])

from sklearn.cross_validation import train_test_split
from keras.utils import np_utils

x_train, x_test, y_train, y_test = train_test_split(text_content, text_title, test_size=0.2, random_state=9487)

# 将每个词用词典中的数值代替
x_train_word_ids = tokenizer.texts_to_sequences(x_train)
x_test_word_ids = tokenizer.texts_to_sequences(x_test)
y_train_word_ids = tokenizer.texts_to_sequences(y_train)
y_test_word_ids = tokenizer.texts_to_sequences(y_test)
# 序列模式
x_train = pad_sequences(x_train_word_ids, maxlen=max_word_length+2, padding='post', truncating='post')
x_test = pad_sequences(x_test_word_ids, maxlen=max_word_length+2, padding='post', truncating='post')
y_train = pad_sequences(y_train_word_ids, maxlen=max_title_length+2, padding='post', truncating='post')
y_test = pad_sequences(y_test_word_ids, maxlen=max_title_length+2, padding='post', truncating='post')
# 序列長度
x_train_size = [len(word_ids) for word_ids in x_train_word_ids]
x_test_size = [len(word_ids) for word_ids in x_test_word_ids]
y_train_size = [len(word_ids) for word_ids in y_train_word_ids]
y_test_size = [len(word_ids) for word_ids in y_test_word_ids]

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Building prefix dict from C:\Users\User\Anaconda3\Lib\site-packages\jieba\dict.txt.big ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.ufa6ae29b0cbce8b45e006c7fa30eaaf8.cache
Loading model cost 0.955 seconds.
Prefix dict has been built succesfully.


199446
1 的 的 479528 108422
2 我 我 187622 60294
3 <GO> <GO> 172800 172800
4 <EOS> <EOS> 172800 172800
5 是 是 157302 68972
6 了 了 129931 60543
7 有 有 108041 59614
8 在 在 97066 54361
9 也 也 86587 48759
10 都 都 75404 45560
11 就 就 71400 42410
12 不 不 66238 42797
13 很 很 54725 34820
14 會 會 54210 36201
15 他 他 49560 20170
16 跟 跟 48539 31439
17 看 看 48476 30452
18 你 你 48168 21707
19 想 想 47770 34168
1000 跳 跳 1246 1022
2000 整合 整合 607 489
3000 往往 往往 366 345
4000 007 007 255 156
5000 處於 處於 192 188
6000 一般來說 一般來說 154 150
7000 何 何 125 101
8000 有鬼 有鬼 105 100
9000 備 備 89 89
10000 半年前 半年前 78 78
11000 掉下來 掉下來 68 66
12000 托爾 托爾 61 46
13000 傻子 傻子 54 51
14000 活得 活得 49 48
15000 寫給 寫給 44 41
16000 希斯 希斯 41 31
17000 毆打 毆打 37 33
18000 編制 編制 34 29
19000 激怒 激怒 32 30
20000 神秘性 神秘性 30 5
21000 睡意 睡意 27 27
22000 真情 真情 26 24
23000 寫些 寫些 24 24
24000 野蠻女友 野蠻女友 23 14
25000 20th 20th 21 17
26000 世家 世家 20 20
27000 天降 天降 19 18
28000 人卡 人卡 18 13
29000 決定權 決定權 17 17
30000 始於 始於 16 15
31000 初號機 初號機 16 10
32000 早睡早起 早睡早起 15 15
33000 記號 記號

C:\Users\User\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [61]:
import os
import time
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.layers.core import Dense
from tensorflow.contrib import layers
################################################################
# Set parameter
################################################################
patience = 0
num_epoch = 60
batch_size = 64
vocab_size = words_limit
limit_generation_len = max_title_length + 2

rnn_size = 150
num_layers = 3
embedding_size = 300
exp_decay_steps = num_instances / batch_size
exp_decay_rate = 0.86

lstm_dropout = 0.5
l2_reg_const = 0.001

lr = 0.05
lr_alpha = 1e-3
cos_decay_steps = num_epoch * 3 / 4 * num_instances / batch_size

clip_const = 100.0
clip_alpha = 1.0

################################################################
# Build seq2seq model
################################################################
def build_model_in():
    ######## model input tensor ########
    is_training = tf.placeholder(tf.bool, shape=(), name='is_training')
    is_decoder_in = tf.placeholder(tf.bool, shape=(), name='is_decoder_in')
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    input_seq_len = tf.placeholder(tf.int32, (None,), name='input_seq_len')
    target_seq_len = tf.placeholder(tf.int32, (None,), name='target_seq_len')
    limit_target_seq_len = tf.placeholder(tf.int32, (), name='limit_target_seq_len')
    return is_training, is_decoder_in, inputs, targets, input_seq_len, target_seq_len, limit_target_seq_len
    
def build_model(is_training, is_decoder_in, inputs, targets, input_seq_len, target_seq_len, limit_target_seq_len):
    max_target_seq_len = tf.reduce_max(target_seq_len, name='max_target_seq_len')
    validation_batch_size = tf.shape(targets)[0]
    start_tokens = tf.tile(tf.constant([vocab[GO_word]], dtype=tf.int32),
                               [validation_batch_size], name='start_tokens')
    
    ######## Embedding for encoder and decoder ########
    embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size]))
    encoder_embed_input = tf.nn.embedding_lookup(embeddings, inputs)
    decoder_embed_input = tf.nn.embedding_lookup(embeddings, targets)
    
    ######## Encoder by LSTM ########
    # LSTM cell
    def get_lstm_cell(rnn_size, layer):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size, 
                                            initializer=tf.random_uniform_initializer(-1.0, 1.0))
        #activity_regularizer=tf.contrib.layers.l2_regularizer(l2_reg_const) 
        if is_training is not None and lstm_dropout < 1.0:
            if layer == 0:
                lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, 
                                                          input_keep_prob=lstm_dropout, 
                                                          output_keep_prob=lstm_dropout)
            else:
                lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, 
                                                          output_keep_prob=lstm_dropout)
        return lstm_cell

    # Struct encoder by LSTM bi-directional multi-layer
    encoder_f_cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size, layer)
                                                  for layer in range(num_layers)])
    encoder_b_cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size, layer)
                                                  for layer in range(num_layers)])
    encoder_bi_outputs, encoder_bi_state = tf.nn.bidirectional_dynamic_rnn(encoder_f_cell, 
                                                                           encoder_b_cell, 
                                                                           encoder_embed_input, 
                                                                           sequence_length=input_seq_len, 
                                                                           dtype=tf.float32)
    encoder_outputs = tf.concat((encoder_bi_outputs[0], encoder_bi_outputs[1]), 2)
    encoder_states = tuple([tf.contrib.rnn.LSTMStateTuple(tf.concat((state_f.c, state_b.c), 1), 
                                                          tf.concat((state_f.h, state_b.h), 1))
                     for state_f, state_b in zip(encoder_bi_state[0], encoder_bi_state[1])])
    
    ######## Decoder by LSTM ########
    # Struct LSTM cell in Decoder
    multi_lstm_cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size*2, layer) 
                                                for layer in range(num_layers)])
    
    # Bahdanau attention
    #attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(rnn_size*2, encoder_outputs,
    attention_mechanism = tf.contrib.seq2seq.LuongAttention(rnn_size*2, encoder_outputs,
                                                            memory_sequence_length=input_seq_len)
    decoder_cell = tf.contrib.seq2seq.AttentionWrapper(multi_lstm_cell, 
                                                       attention_mechanism, 
                                                       attention_layer_size=num_layers)
    #decoder_cell = tf.contrib.rnn.OutputProjectionWrapper(attn_cell, slot_size)
    
    # Output fully-connecting layer
    output_layer = Dense(vocab_size, 
                         kernel_initializer=tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
        # Struct predicting decoder (Share variable with training)
    with tf.variable_scope('decode'):
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                     start_tokens, 
                                                                     vocab[EOS_word])
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, predicting_helper,
                                                             decoder_cell.zero_state(batch_size, tf.float32).clone(cell_state=encoder_states), 
                                                             output_layer)
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(predicting_decoder,
                                                                            impute_finished=True,
                                                                            maximum_iterations=limit_target_seq_len)
    
    # Struct training decoder (Share variable with predicting)
    with tf.variable_scope('decode', reuse=True):
        if is_decoder_in is not None:
            training_helper = tf.contrib.seq2seq.TrainingHelper(decoder_embed_input,
                                                                target_seq_len)
            training_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, training_helper,
                                                               decoder_cell.zero_state(batch_size, tf.float32).clone(cell_state=encoder_states), 
                                                               output_layer) 
            training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                               impute_finished=True,
                                                                               maximum_iterations=max_target_seq_len)
        else:
            training_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                     start_tokens, 
                                                                     vocab[EOS_word])
            training_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, training_helper,
                                                               decoder_cell.zero_state(batch_size, tf.float32).clone(cell_state=encoder_states), 
                                                               output_layer) 
            training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                               impute_finished=True,
                                                                               maximum_iterations=max_target_seq_len)
    
    ######## Model output tensor ########
    training_logits = tf.identity(training_decoder_output.rnn_output, name='trainings')
    predicting_logits = tf.identity(predicting_decoder_output.sample_id, name='predictions')
    return training_logits, predicting_logits, max_target_seq_len
    
def build_optimizer(training_logits, predicting_logits, targets, target_seq_len, max_target_seq_len):
    ######## Optimization ########
    if is_training is not None:
        with tf.name_scope('optimization'):
            masks = tf.sequence_mask(target_seq_len, max_target_seq_len, dtype=tf.float32)
            global_step = tf.Variable(0, trainable=False)
            l2 = tf.Variable(0.0, trainable=False)
            # Loss function
            cost = tf.contrib.seq2seq.sequence_loss(training_logits, targets, masks, name='sequence_cost')
            # Add regularization to Loss
            l2 = l2_reg_const * sum( tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables() if not ('Bias' in tf_var.name))
            #regularization_loss = tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
            cost = cost + l2
            # Optimizer
            learning_rate = tf.train.exponential_decay(lr, global_step, exp_decay_steps, 
                                                       exp_decay_rate, staircase=True)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            
            # Gradient Clipping
            clip_rate = tf.train.cosine_decay(clip_const, global_step, cos_decay_steps, alpha=clip_alpha)
            gradients, vriables = zip(*optimizer.compute_gradients(cost))
            gradients = [None if gradient is None else tf.clip_by_norm(gradient, clip_rate) for gradient in gradients]
            global_step = global_step + 1
            train_step = optimizer.apply_gradients(zip(gradients, vriables), name='train_step')
    return cost, train_step


################################################################
# Prepare to run session
################################################################
num_instances = len(y_train)
iter_per_epoch = int(num_instances / batch_size)
test_num_instances = len(y_test)
test_iter_per_epoch = int(test_num_instances / batch_size)

only_train_helper = False
total_start_t = time.time()
build_new_model = 1
if build_new_model:
    str_epoch = 0
    model_name = 'PTT_Encoder-Decoder_model_fixed_' + str(total_start_t)
else:
    str_epoch = 30
    model_name = 'PTT_Encoder-Decoder_model_fixed_1522421878.8546176'

iter_pct10 = int(iter_per_epoch / 10)
if (num_instances % batch_size) > 0:
    iter_per_epoch += 1
batch_cutoff = [0]
for i in range(iter_per_epoch - 1):
    batch_cutoff.append(batch_size * (i+1))
batch_cutoff.append(num_instances)

best_validation_loss = 0.0
early_stop_counter = 0

mdl_dir = os.path.join(base_dir, 'model')
if not os.path.exists(mdl_dir):
    os.makedirs(mdl_dir)
model_dir = os.path.join(mdl_dir, model_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

################################################################
# Run model session
################################################################
train_graph = tf.Graph()
if build_new_model:
    ######## Build model parameters ########
    with train_graph.as_default():
        # Build input parameters
        is_training, is_decoder_in, inputs, targets, input_seq_len, target_seq_len, limit_target_seq_len = build_model_in()
        # Build model and  weight parameters
        training_logits, predicting_logits, max_target_seq_len = build_model(is_training, is_decoder_in,
                                                                             inputs, targets, 
                                                                             input_seq_len, target_seq_len, 
                                                                             limit_target_seq_len)
        # Build optimizer
        cost, train_step = build_optimizer(training_logits, predicting_logits,
                                           targets, target_seq_len, max_target_seq_len)

with tf.Session(graph=train_graph) as sess:
    if build_new_model:
        sess.run(tf.global_variables_initializer())
        best_epoch = 1
    else:
        ######## Load model parameters ########
        # Load model and weight parameters
        best_epoch = str_epoch
        load_model_dir = os.path.join(model_dir, 'model-{}'.format(str_epoch))
        loader = tf.train.import_meta_graph(load_model_dir + '.meta')
        loader.restore(sess, load_model_dir)
        # Load named input parameters
        is_training = train_graph.get_tensor_by_name('is_training:0')
        inputs = train_graph.get_tensor_by_name('inputs:0')
        targets = train_graph.get_tensor_by_name('targets:0')
        input_seq_len = train_graph.get_tensor_by_name('input_seq_len:0')
        target_seq_len = train_graph.get_tensor_by_name('target_seq_len:0')
        limit_target_seq_len = train_graph.get_tensor_by_name('limit_target_seq_len:0')
        # Load named output and optimizer parameters
        training_logits = train_graph.get_tensor_by_name('trainings:0')
        predicting_logits = train_graph.get_tensor_by_name('predictions:0')
        cost = train_graph.get_tensor_by_name('optimization/sequence_cost/truediv:0')
        train_step = train_graph.get_operation_by_name('optimization/gradients_1')
        
    ######## Run model epoch ########
    decoder_global_step = 0
    for e in range(str_epoch,str_epoch+num_epoch):
        rand_idxs = np.random.permutation(num_instances)
        print('\n========')
        print('epoch' + str(e+1))
        print('========')
        start_t = time.time()
        
        ######## Get random input batch and training model. ########
        for i in range(iter_per_epoch):
            X_batch = []
            Y_batch = []
            X_size = []
            Y_size = []
            for n in range(batch_cutoff[i],batch_cutoff[i+1]):
                X_batch.append(x_train[rand_idxs[n]])
                Y_batch.append(y_train[rand_idxs[n]])
                X_size.append(x_train_size[rand_idxs[n]])
                Y_size.append(y_train_size[rand_idxs[n]])
            max_X_size = max(X_size)
            max_Y_size = max(Y_size)
            X_batch = np.array(X_batch)
            Y_batch = np.array(Y_batch)
            X_size = np.array(X_size)
            Y_size = np.array(Y_size)
            
            # Select helper based on learning time.
            decoder_random = np.random.uniform(0.0, 1.0)
            helper_rate = np.power(exp_decay_rate,
                                   (float(decoder_global_step / exp_decay_steps)))
            decoder_global_step +=1
            if decoder_random<helper_rate or only_train_helper:
                _, loss = sess.run( [train_step, cost],
                                    {is_training: True, is_decoder_in: True,
                                     inputs: X_batch[:,:max_X_size], targets: Y_batch[:,:max_Y_size],
                                     input_seq_len: X_size, target_seq_len: Y_size,
                                     limit_target_seq_len: limit_generation_len})
            else:
                _, loss = sess.run( [train_step, cost],
                                    {is_training: True,
                                     inputs: X_batch[:,:max_X_size], targets: Y_batch[:,:max_Y_size],
                                     input_seq_len: X_size, target_seq_len: Y_size,
                                     limit_target_seq_len: limit_generation_len})
            
            if i % iter_pct10 == iter_pct10 - 1:
                print('Iteration ',i+1,' (',round(i*100/iter_per_epoch,2),'%) Train Loss: ',
                      loss,'; run time[s]: ', time.time() - start_t,end='\n')
        print('Train Loss: ',loss)
        
        ######## Generate title form testing content and validating model. ########
        i_str = random.randint(0, test_num_instances+batch_size-1)
        i_end = i_str + batch_size
        X_batch = np.array(x_test[i_str:i_end])
        Y_batch = np.array(y_test[i_str:i_end])
        X_size = np.array(x_test_size[i_str:i_end])
        Y_size = np.array(y_test_size[i_str:i_end])
        max_X_size = max(X_size)
        max_Y_size = max(Y_size)
        answer_logits, validation_loss = sess.run([predicting_logits, cost], 
                                {inputs: X_batch[:,:max_X_size], targets: Y_batch[:,:max_Y_size], 
                                 input_seq_len: X_size, target_seq_len: Y_size,
                                 limit_target_seq_len: limit_generation_len})
        output_index = random.randint(0, batch_size-1)
        print('Test loss: ',validation_loss)
        print('Target Words: {}'.format(" ".join([anti_vocab[i] for i in Y_batch[output_index] if i != 0])))
        print('Response Words: {}'.format(" ".join([anti_vocab[i] for i in answer_logits[output_index] if i != 0])))
        
        ######## Save model weight parameters and information. ########
        if validation_loss < best_validation_loss or e == 0:
            best_validation_loss = validation_loss
            best_epoch = e
            early_stop_counter = 0
            print('Save best score!! '+str(best_validation_loss))
        else:
            early_stop_counter += 1
            print('Count early stop!! '+str(early_stop_counter))

        print('Elapsed time in epoch ' + str(e+1) + ': ' + str(time.time() - start_t) + ' [s]')

        model_path = os.path.join(model_dir, 'model-%d' %(e+1))
        saver = tf.train.Saver()
        saver.save(sess, model_path)
        
        if patience != 0 and early_stop_counter >= patience:
            print('\n#######')
            print('Best model')
            print('#######')
            print('Stop by early stopping')
            print('Best score: ', best_validation_loss, 'Beat model: ', best_epoch)
            break

print('\nElapsed time in total: ' + str(time.time() - total_start_t))


epoch1
Iteration  108  ( 9.91 %) Train Loss:  3584.6252 ; run time[s]:  88.83303356170654
Iteration  216  ( 19.91 %) Train Loss:  3638.8127 ; run time[s]:  172.73257040977478
Iteration  324  ( 29.91 %) Train Loss:  1363.8794 ; run time[s]:  255.29424905776978
Iteration  432  ( 39.91 %) Train Loss:  914.90485 ; run time[s]:  336.4072632789612
Iteration  540  ( 49.91 %) Train Loss:  787.9465 ; run time[s]:  422.2752254009247
Iteration  648  ( 59.91 %) Train Loss:  722.5763 ; run time[s]:  506.81256914138794
Iteration  756  ( 69.91 %) Train Loss:  661.82336 ; run time[s]:  588.8499355316162
Iteration  864  ( 79.91 %) Train Loss:  611.68756 ; run time[s]:  663.4598069190979
Iteration  972  ( 89.91 %) Train Loss:  568.998 ; run time[s]:  737.7987787723541
Iteration  1080  ( 99.91 %) Train Loss:  531.05707 ; run time[s]:  814.0470354557037
Train Loss:  531.05707
Test loss:  530.66077
Target Words: <GO> 不 懂 男人 付錢 的 哲學 <EOS>
Response Words: <GO> <EOS>
Save best score!! 530.66077
Elapsed time 

Iteration  1080  ( 99.91 %) Train Loss:  6.597954 ; run time[s]:  744.2923345565796
Train Loss:  6.597954
Test loss:  6.3414555
Target Words: <GO> 情色 語言 51 個 練習 <EOS>
Response Words: <GO> 有 大家 有 大家 女生 要 吧 吧 吧 結局 <EOS>
Save best score!! 6.3414555
Elapsed time in epoch 8: 744.7117927074432 [s]

epoch9
Iteration  108  ( 9.91 %) Train Loss:  6.2407084 ; run time[s]:  74.50417304039001
Iteration  216  ( 19.91 %) Train Loss:  6.238619 ; run time[s]:  147.88773584365845
Iteration  324  ( 29.91 %) Train Loss:  6.301245 ; run time[s]:  221.6898455619812
Iteration  432  ( 39.91 %) Train Loss:  6.650647 ; run time[s]:  295.5168282985687
Iteration  540  ( 49.91 %) Train Loss:  6.357115 ; run time[s]:  369.6488540172577
Iteration  648  ( 59.91 %) Train Loss:  6.2488604 ; run time[s]:  443.087144613266
Iteration  756  ( 69.91 %) Train Loss:  6.3139377 ; run time[s]:  516.1367902755737
Iteration  864  ( 79.91 %) Train Loss:  6.1294336 ; run time[s]:  589.6324305534363
Iteration  972  ( 89.91 %) Train

Iteration  540  ( 49.91 %) Train Loss:  6.4185696 ; run time[s]:  366.93106055259705
Iteration  648  ( 59.91 %) Train Loss:  6.462558 ; run time[s]:  439.8653063774109
Iteration  756  ( 69.91 %) Train Loss:  6.345972 ; run time[s]:  513.1430485248566
Iteration  864  ( 79.91 %) Train Loss:  6.1901536 ; run time[s]:  586.0101752281189
Iteration  972  ( 89.91 %) Train Loss:  6.1569614 ; run time[s]:  658.778814792633
Iteration  1080  ( 99.91 %) Train Loss:  6.205734 ; run time[s]:  731.9636716842651
Train Loss:  6.205734
Test loss:  6.1291246
Target Words: <GO> 大家 有 泡澡 跑 皮 的 經驗 嗎 <EOS>
Response Words: <GO> 的 的 <GO> 台南 怎麼辦 請益 請益 請益 請益 請益 請益 請益 請益 怎麼辦 怎麼辦 請益 請益 請益 請益 台南 請益
Count early stop!! 4
Elapsed time in epoch 16: 732.3537392616272 [s]

epoch17
Iteration  108  ( 9.91 %) Train Loss:  6.338465 ; run time[s]:  73.09900736808777
Iteration  216  ( 19.91 %) Train Loss:  6.29216 ; run time[s]:  145.86541271209717
Iteration  324  ( 29.91 %) Train Loss:  6.4639297 ; run time[s]:  218.9007320404


epoch24
Iteration  108  ( 9.91 %) Train Loss:  6.5624304 ; run time[s]:  73.19814991950989
Iteration  216  ( 19.91 %) Train Loss:  6.560229 ; run time[s]:  146.46801924705505
Iteration  324  ( 29.91 %) Train Loss:  6.607561 ; run time[s]:  219.69314289093018
Iteration  432  ( 39.91 %) Train Loss:  5.9705553 ; run time[s]:  293.3555347919464
Iteration  540  ( 49.91 %) Train Loss:  6.459579 ; run time[s]:  367.1367223262787
Iteration  648  ( 59.91 %) Train Loss:  8.270302 ; run time[s]:  440.2074592113495
Iteration  756  ( 69.91 %) Train Loss:  6.733115 ; run time[s]:  513.807021856308
Iteration  864  ( 79.91 %) Train Loss:  5.9446874 ; run time[s]:  587.345196723938
Iteration  972  ( 89.91 %) Train Loss:  6.512391 ; run time[s]:  659.9624149799347
Iteration  1080  ( 99.91 %) Train Loss:  6.4126487 ; run time[s]:  733.5748996734619
Train Loss:  6.4126487
Test loss:  6.441737
Target Words: <GO> 這些 行為 算是 有 喜歡 嗎 我 <EOS>
Response Words: <GO> <GO> 台南 <GO> 周末 請益 請益 請益 台南 請益 請益 <GO> 請益 請益 請益 請

In [63]:
import os
import random
################################
#Generate title form testing content and validating model. 
################################
base_dir = 'C:/Users/User/Raw data/PTT'
model_name = 'PTT_Encoder-Decoder_model_fixed_1522688220.0745473'
epoch = 26
model_path = os.path.join(base_dir, 'model', model_name, 'model-{}'.format(epoch))
print('Test epoch: ', epoch)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load model
    loader = tf.train.import_meta_graph(model_path + '.meta')
    loader.restore(sess, model_path)
    # Named parameters
    #is_training = loaded_graph.get_tensor_by_name('is_training:0')
    inputs = loaded_graph.get_tensor_by_name('inputs:0')
    targets = loaded_graph.get_tensor_by_name('targets:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    input_seq_len = loaded_graph.get_tensor_by_name('input_seq_len:0')
    target_seq_len = loaded_graph.get_tensor_by_name('target_seq_len:0')
    limit_target_seq_len = loaded_graph.get_tensor_by_name('limit_target_seq_len:0')
    
    for _ in range(8):
        i_str = random.randint(0,test_num_instances-1)
        i_end = i_str + 1

        X_batch = np.tile(x_test[i_str], (batch_size, 1))
        Y_batch = np.tile(y_test[i_str], (batch_size, 1))
        X_size = np.tile(x_test_size[i_str], (batch_size))
        Y_size = np.tile(y_test_size[i_str], (batch_size))
        max_X_size = x_test_size[i_str]
        max_Y_size = y_test_size[i_str]
        answer_logits = sess.run( logits, {inputs: X_batch, targets: Y_batch, 
                                             input_seq_len: X_size, target_seq_len: Y_size,
                                             limit_target_seq_len: limit_generation_len})

        output_index = 0 #random.randint(0,batch_size-1)
        print('\n========')
        print('Test index: ', i_str + output_index)
        print('========')
        print('Test input Words: {}'.format(" ".join([anti_vocab[i] for i in X_batch[output_index] if i != 0])))
        print('Target Words: {}'.format(" ".join([anti_vocab[i] for i in Y_batch[output_index] if i != 0])))
        print('Response Words: {}'.format(" ".join([anti_vocab[i] for i in answer_logits[output_index] if i != 0])))

Test epoch:  26
INFO:tensorflow:Restoring parameters from C:/Users/User/Raw data/PTT\model\PTT_Encoder-Decoder_model_fixed_1522688220.0745473\model-26

Test index:  471
Test input Words: <GO> 代 po 原 po 女生 請問 你們 會 因為 女生 喜歡 看 鄉土 扣分 嗎 例如 天下 女人 心 天下第一 味 風水 世家 等 節目 只要 撥出 時間 一定 會 看 室友 要 看 棒球 經典 賽 的 時候 我 還是 堅持 要 看 這些 節目 請問 這樣 在 男生 扣分 嗎 <EOS>
Target Words: <GO> 你們 會 因為 女生 看 鄉土 扣分 嗎 <EOS>
Response Words: <GO> 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的

Test index:  485
Test input Words: <GO> 2015 年 09 月 08 日 眼鏡 男 在 麥當勞 手槍 過程 被 其他 顧客 拍 下 張姓 讀者 提供 動 新聞 綜合 報導 有 這麼 猴急 嗎 逢甲大學 一群 男女 學生 前晚 到 麥當勞 用餐 發現 一名 男子 將 雙手 伸進 褲襠 內 不斷 上下 很 明顯 是 在 打 手槍 雙手 伸 出來 後 趴在 桌上 睡覺 同樣 的 舉動 在 3 個 小時 內 重覆 好 幾次 一名 男 大生 拍 下 畫面 向 蘋果 爆料 同行 女 大生 批 好 噁心 台中市 第六 分局 西屯 派出所 看過 影片 後 表示 該名 男子 雖有 打 手槍 的 動作 但 隔 著 褲子 未 露出 下體 恐 不 符合 刑法 公然 猥褻罪 中 意圖 供 人 的 要件 且 他 未 對 著 旁人 進行 猥褻 動作 也 不 符合 性騷擾 要件 但 建議 民眾 發現 時 可 立即 向 警方 報案 由 警員 到場 勸導 若 勸導 不 聽 可依 社會秩序 維護 法 第 83 條 在 公共場所 為 放蕩 之 姿勢 而 有 妨害 善良 風俗 處以 6000 元 以下 罰 鍰 下面 癢 不能 抓 嗎 莫名其妙 <EOS>
Target Words: <GO>